# Title: Classifying Explicit Songs on Spotify From 2010-2019 via K-Nearest Neighbors

In [7]:
install.packages("themis")
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)
library(themis)
library(RColorBrewer)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



ERROR: Error: package or namespace load failed for ‘tidyverse’ in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace ‘rlang’ 0.4.7 is already loaded, but >= 1.0.2 is required


# 1. Introduction

### In this study, we will study a data set of Top Hit Spotify Songs from 2000-to 2019, originating from Kaggle. 
The data is organized into 18 columns, each of which describes the track and its quality. A detailed description of all the columns content can be found here: https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019".

We want to answer a predictive question: Can we use a song's traits to predict whether a song in 2010-2019 on Spotify is explicit or not? Answering this question is noteworthy because Spotify has an explicit content filter that allows users to filter out songs containing language that may be considered offensive to children. However, Spotify's explicit content tag is based on information provided by the rights holder; thus, not all tracks with explicit content are properly labelled, and users must manually report those they missed. Hence, this study set out to investigate an alternative solution to predict and thus label whether a song may be explicit or not. A rigorous answer to this question could help Spotify enhance its explicit content filter and improve user experience.


In [ ]:
# read + clean + wrangle data ----------------------------------------------------
single_genre <- c('country', 'latin', 'Dance/Electronic', 'Folk/Acoustic',
'pop', 'rock','hip hop','R&B','metal','jazz','blues','classical','World/Traditional')

In [ ]:
set.seed(999)
options(repr.plot.width = 8, repr.plot.height = 8)
song <- read_csv('https://raw.githubusercontent.com/nicolelassetter/DSCI100-project-g28/main/songs_normalize.csv') %>%
    mutate(genre <- as_factor(genre))%>%
    filter(genre %in% single_genre)
head(song) # small preview of data set

In [ ]:
# count the row of missing data to see if we need to tidy our data, and the result is zero.
sum(anyNA(song))

# count how many explicit vs non-explict are present to see if we need to balance the data prior training to avoid bias,
# and the result is that this data is imbalance, and will need to be balanced prior training.
n_explicit <- song %>%
    group_by(explicit) %>%
    summarize(n = n())
n_explicit

In [ ]:
# split data to Training + Testing Data ---------------------------------------------
song_split <- initial_split(song, prop = 0.75, strata = explicit)
song_train <- training(song_split)
track_test <- testing(song_split)

In [ ]:
#summarize the whole training data
summary(song_train)

In [ ]:
## Plots that shows little to no relationship to explicitness, put here to show that we did all the exploration, but since there are 
## too many graphs, we did not show all of them.
p00 <- ggplot(song_train,aes(x = popularity,color = explicit))+geom_bar()+scale_fill_brewer(palette = "Set1")
p01 <- ggplot(song_train,aes(x = year,color = explicit))+geom_bar()+scale_fill_brewer(palette = "Set1")
p02 <- ggplot(song_train,aes(x = energy ,color = explicit))+geom_bar()+scale_fill_brewer(palette = "Set1")
p03 <- ggplot(song_train,aes(x = loudness,color = explicit))+geom_bar()+scale_fill_brewer(palette = "Set1")
p04 <- ggplot(song_train,aes(x = mode ,color = explicit))+geom_bar()+scale_fill_brewer(palette = "Set1")
p05 <- ggplot(song_train,aes(x = valence ,color = explicit))+geom_histogram()
p06 <- ggplot(song_train,aes(x = danceability ,color = explicit))+geom_histogram()
p07 <- ggplot(song_train,aes(x = popularity ,color = explicit))+geom_histogram()
p08 <- ggplot(song_train,aes(x = key ,color = explicit))+geom_histogram()
p09 <- ggplot(song_train,aes(x = loudness ,color = explicit))+geom_histogram()
p10 <- ggplot(song_train,aes(x = acousticness ,color = explicit))+geom_histogram()
p11 <- ggplot(song_train,aes(x = instrumentalness ,color = explicit))+geom_histogram()
p12 <- ggplot(song_train,aes(x = liveness ,color = explicit))+geom_histogram()
p13 <- ggplot(song_train,aes(x = valence ,color = explicit))+geom_histogram()
p14 <- ggplot(song_train,aes(x = tempo ,color = explicit))+geom_histogram()
p15 <- ggplot(song_train,aes(x = genre ,color = explicit))+geom_bar()

In [ ]:
# summarise only training data info into tables ----------------------------------------
n_genre_table_training <- song_train %>%
    group_by(genre, explicit) %>%
    summarize(n = n()) 
n_genre_table_training

In [ ]:
mean_table_training <- song_train %>%
    select(speechiness, acousticness, energy, danceability) %>%
    map_df(mean)
mean_table_training

# 2. Methods

For the sake of the project, we will only use songs with a single genre. 

Data analysis will be conducted on Jupyter notebook using R. We will use the tidyverse, repr, dplyr, RColorBrewer and themis libraries as they contain the functions required for our calculations and visualizations. Our primary method is classification using K-nearest neighbours (KNN). We mostly based our model from Data Science: A First Introduction (Timbers et al., 2022) and accessed our data from Kaggle (Koverha, 2022). To begin, we will create a training and test data set and use only the training data for the preprocessing/preliminary steps.


We split our data set so that 75% of the original data set ends up in the training set, and 25% would be in the test set(using the function initial_split). This includes scaling all predictors (step_scale and step_center) to ensure they have a mean of 0 and a standard deviation of 1. To increase the readability of the 18 columns, we wil only keep the classifier and the predictors.

Hence, the remaining columns from the data set are as follows:

1. explicit: The song or music video contains content that is considered offensive or inappropriate for children.
2. speechiness: Measure from 0.0 to 1.0 of the presence of spoken words. The closer the song is near 1.0, the more words has.
3. energy: Measure from 0.0 to 1.0, a perceptual measure of intensity and activity.
4. danceability: How suitable the track is for dancing based on tempo, rhythm stability, beat strength, and overall regularity. 0 = least danceable, 1.0 = most danceable.
5. genre : Track’s genre.



# To Begin:

We plotted the genre into bar graphs to visualize the genre distribution of our training data. We then used scatter plots to determine the predictors by looking for relationships. Next, we plotted numerous graphs using our training data with explicit as our classifier to discover relationships. We found that speechiness, energy, danceability and genre had a sort of linear or grouped relationship to explicitness through plotting, and the others did not have a relation.

In [ ]:
# Using training data: visualise data to find possible predictors relevant for analysis ----

# Bar: genre vs count (Number of Songs Total)
plot1 <- n_genre_table_training %>%
    ggplot(aes(x = genre, fill = explicit, y = n)) +
    geom_bar(stat = 'identity') +
    labs(x = "Single Song Genres", y = 'Number of Songs Total') +
    ggtitle('Single Song Genres vs Number of Songs Total Colored by Explicit')
plot1

In [ ]:
# Scatter: Speechiness vs Energy Colored and Shaped by Explicit
plot2 <- song_train %>%
    ggplot(aes(x = energy, y = speechiness, colour = explicit, shape = explicit)) +
    geom_point() +
    labs(x = "Energy (Scale from 0-1)", y = 'Speechiness (Scale from 0-1)') +
    ggtitle('Speechiness vs Energy Colored and Shaped by Explicit')
plot2

In [ ]:
# Scatter: Speechiness vs Danceability Colored and Shaped by Explicit
plot3 <- song_train %>%
    ggplot(aes(x = danceability, y = speechiness, colour = explicit, shape = explicit)) +
    geom_point() +
    labs(x = "Danceability (Scale from 0-1)", y = 'Speechiness (Scale from 0-1)') +
    ggtitle('Speechiness vs Danceability Colored and Shaped by Explicit')
plot3

In [ ]:
# Scatter : Speechiness vs Acousticness Colored and Shaped by Explicit
plot4 <- song_train %>%
    ggplot(aes(x = acousticness, y = speechiness, colour = explicit, shape = explicit)) +
    geom_point() +
    labs(x = "Danceability (Scale from 0-1)", y = 'Speechiness (Scale from 0-1)') +
    ggtitle('Speechiness vs Acousticness Colored and Shaped by Explicit')
plot4

### Modeling

The first step of our analysis is selecting the optimal K value for our K-nearest-neighbours (KNN) calculations. The begins with performing a cross-validation calculation using vfold, where we split our overall training data into C evenly sized chunks and perform a 5-fold cross-validation. We then run cross-validation on each train/validation split. This reduces the influence of any one (un)lucky validation set on the estimate. 


In [ ]:
# Perform the data analysis -----------------------------------------------------------------------------------

# select only the columns that contains the response variable + predictors we will use
songs <- song %>%
    select(explicit, speechiness, danceability, energy) %>%
    mutate(explicit = as_factor(explicit))

# split data to Training + Testing data 
songs_split <- initial_split(songs, prop = 0.75, strata = explicit)
songs_train <- training(songs_split)
tracks_test <- testing(songs_split)

Here, we create a recipe that specifies our class label (explicit) and our predictors (the rest of our predictors). We use our training data, as we don't want our testing data to "see light" yet.

In [ ]:
# balancing the data(since there are way too many unexplicit songs)

s_recipe <- recipe(explicit ~ speechiness + danceability + energy, data = songs_train) %>%
    step_upsample(explicit, over_ratio = 1, skip = FALSE) %>%
    prep()
upsampled_songs <- bake(s_recipe, songs_train)
songs_train <- upsampled_songs 
songs_train
upsampled_songs %>%
  group_by(explicit) %>%
  summarize(n = n())


Here, we also perform the preprocessing steps, passing the training data as the data argument in the recipe. Because we are looking for K, we say neighbors = tune() instead of a number value. 

In [ ]:
# finding optimal K with balanced training data
songs_recipe <- recipe(explicit ~ ., data = songs_train ) %>%
    step_upsample(over_ratio = 1, skip = FALSE) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors()) 


In [ ]:
songs_vfold <- vfold_cv(songs_train, v = 5, strata = explicit)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
    set_engine("kknn") %>%
    set_mode("classification")

k_vals <- tibble(neighbors = seq(from = 1, to = 15, by = 1))

knn_results <- workflow() %>%
    add_recipe(songs_recipe) %>%
    add_model(knn_spec) %>%
    tune_grid(resamples = songs_vfold, grid = k_vals) %>%
    collect_metrics()


Here, we plot our filtered knn_results to visualize the accuracy across different values of K. The K value we chose is K=9 because from the plot, we can see this has the highest accuracy and does not change much when we change K to a nearby value.

In [ ]:
accuracies <- knn_results %>%
     filter(.metric == 'accuracy')

cross_val_plot <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = 'Neighbors', y = 'Accuracy Estimate') +
    theme(text = element_text(size = 20)) +
    scale_x_continuous(breaks = seq(0, 30)) +
    ggtitle( "Figure 5. Estimated Accuracy vs Neighbors") +
    theme(plot.title = element_text(hjust = 0.5))

cross_val_plot

Here, we made a new model specification for our chosen K = 9 and retrained the classifier using the fit function. Finally, we evaluate the estimated accuracy and predict with our test data.

In [ ]:
# build/train the model 
knn_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = 13) %>%
    set_engine("kknn") %>%
    set_mode("classification")

songs_fit <- workflow() %>%
             add_recipe(songs_recipe) %>%
             add_model(knn_spec_2) %>%
            fit(data = tracks_test)


# Results

 
Below are our visualizations of our analysis. The first is a plot of our confusion matrix distributions as a bar graph. The second is a graph summarizing our data analysis findings


In [ ]:
# predict the model accuracy
songs_predictions <- predict(songs_fit, tracks_test) %>%
                        bind_cols(tracks_test)

accuracy_on_test <- songs_predictions %>% 
                        metrics(truth = explicit, estimate = .pred_class) %>%
                        filter(.metric == "accuracy")

print('Table 1: Statistics About The Quality of the Model')
accuracy_on_test


In [ ]:
# confusion matrix 
songs_conf_mat <- songs_predictions %>%
                        conf_mat(truth = explicit, estimate = .pred_class) 

print('Table2: Confusion Matrix of The Classifier')
songs_conf_mat


In [ ]:
# visualization of the analysis ----------------------------------------------------------------------------

#bar graph of confusion matrix

mat_graph <- songs_predictions %>%
    group_by(explicit)%>%
    ggplot(aes(x = explicit,color = .pred_class))+geom_bar()+labs(x = "Explicity", y = 'Number of Songs') +
    ggtitle('Figure 6. Bar graph of Confusion Matrix') +
    theme(plot.title = element_text(hjust = 0.5))
mat_graph


In [ ]:
# Table with best K, variables, accuracy
summary_of_analysis = matrix(c('Explicit', 'Speechiness, danceability, energy', 'K = 13', '85.05 %'), ncol = 1, byrow = TRUE) 

rownames(summary_of_analysis) <- c('Response variable', 'Predictors used', 'Optimal K', "Classifier's estimated accuracy on test data")
colnames(summary_of_analysis) <- c('Findings')

analysis <- as.data.frame(summary_of_analysis) 
analysis

In [ ]:
library(cowplot)

correct_plot <- ggplot(songs_predictions, aes(x = energy, y = speechiness, color = explicit)) +
                  geom_point() +
                  labs(x = 'Energy(standardized)', y = 'Speechiness (standardized)', color = 'Explicit label') +
                  theme(text = element_text(size = 10)) 

predict_plot <- ggplot(songs_predictions, aes(x = energy, y = speechiness, color = .pred_class)) +
                  geom_point() +
                  labs(x = 'Energy(standardized)', y = 'Speechiness (standardized)', color = 'Explicit label') +
                  theme(text = element_text(size = 10)) 

plot_grid(correct_plot,predict_plot, ncol = 2, NULL)



## *Work Cited*

(1) https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019

(2) https://community.spotify.com/t5/Content-Questions/Explicit-content/td-p/4625150

(3) https://datasciencebook.ca/
